In [1]:
import pandas as pd

In [2]:
df2020 = pd.read_csv('survey_results_public_2020.csv')
df2020_filter = df2020.loc[:, [
    'DevType',
    'LanguageWorkedWith', 
    'LanguageDesireNextYear', 
    'WebframeWorkedWith', 
    'WebframeDesireNextYear', 
    'DatabaseWorkedWith',
    'DatabaseDesireNextYear',
    'MiscTechWorkedWith',
    'MiscTechDesireNextYear',
    'PlatformWorkedWith',
    'PlatformDesireNextYear',
]]
df2020_filter['year_of_survey'] = 2020

In [3]:
display(df2020_filter.head())

,DevType,LanguageWorkedWith,LanguageDesireNextYear,WebframeWorkedWith,WebframeDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,year_of_survey
0,"Developer, desktop or enterprise applications;...",C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,ASP.NET;ASP.NET Core,ASP.NET Core,Elasticsearch;Microsoft SQL Server;Oracle,Microsoft SQL Server,.NET;.NET Core,.NET Core;Xamarin,Windows,Android;iOS;Kubernetes;Microsoft Azure;Windows,2020
1,"Developer, full-stack;Developer, mobile",JavaScript;Swift,Python;Swift,NaN,NaN,NaN,NaN,React Native,React Native;TensorFlow;Unity 3D,iOS,iOS;Kubernetes;Linux;MacOS,2020
2,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,NaN,HTML/CSS;Ruby;SQL,Java;Ruby;Scala,Ruby on Rails,Django;Ruby on Rails,MySQL;PostgreSQL;Redis;SQLite,MySQL;PostgreSQL,Ansible,Ansible;Chef,AWS;Docker;Linux;MacOS;Windows,Docker;Google Cloud Platform;Heroku;Linux;Windows,2020
